In [1]:
import sys
from pathlib import Path

pyswap_path = Path("/home/zawadzkim/projects/pySWAP")
sys.path.append(str(pyswap_path.resolve()))


## Create a database connection

Storing data in the database instead instead of individual files would make it easier to manage multiple models and their versions.

In [2]:
from pyswap.core.database import connection as con

db_path = Path(pyswap_path, "pyswapdb.sqlite")
con.DatabaseConnection(db_path)


Database exists. Connecting...
Table data_swaprun_association does not exist in the database. Creating...
Table data exists in the database.
Table swapmodel exists in the database.


## Metadata

Metadata is an important part of a project. Attached to the model definition provides some basic information about who created the model and how to contact them.

In [4]:
from pyswap.core.metadata import Metadata

meta = Metadata(author="John Doe",
                institution="University of Somewhere",
                email="john.doe@somewhere.com",
                project_name="Test",
                swap_ver="4.0")

## Simulation settings

The first thing to do is to define the simulation settings. The `SimSettings` object will store the information about the environment, files the user wants to output, simulation period, etc. 

In [25]:
from pyswap.core.simsettings import SimSettings
from datetime import date as dt

simset = SimSettings(tstart='2018-01-01', 
                     swyrvar=1,
                     tend=dt(2018, 1, 2),
                     outdat=[dt(2018, 1, 1), dt(2018, 1, 1)],
                     inlist_csv=['rain', 'evap'],
                     dznew=[0.13452, 0.2345, 0.333])


## Meteorology

Usually a SWAP model requires some form of meteorological data in the form of .met file. For simplicity, we use a csv file (with .met extension) containing data from multiple years. One of the implemented methods fetches the data from KNMI API.

In [4]:
from pyswap.atmosphere.meteorology import MeteorologicalData

meteo_data = MeteorologicalData(metadata=meta)
meteo_data.weather_kmni()

In [5]:
from pyswap.atmosphere.meteorology import PenmanMonteith
pen_mon = PenmanMonteith(alt=100.0)

In [6]:
from pyswap.atmosphere.meteorology import Meteorology, PenmanMonteith
from pandas import DataFrame

rainflux = DataFrame({'time': [1.0, 360.0], 
            'rainflux': [20.0, 20.0]})

meteo = Meteorology(metfil='377',
                    meteo_data=meteo_data,
                    lat=43.0,
                    swetr=True,
                    penman_monteith=pen_mon,
                    swdivide=False,
                    swetsine=False,
                    swrain=0,
                    rainflux=rainflux)



## Crop

In [7]:
from pyswap.plant.crop import CropFile

crop = CropFile(name='maizes', path='../../data/.crp/maizes.crp')

In [8]:
from pyswap.plant.crop import Crop
from pandas import DataFrame
from datetime import date as dt
# Would be nice to have a subclass for managing the crop files

croprotation = DataFrame({'cropstart':[dt(2018, 1, 1), dt(2018, 1, 1), dt(2018, 1, 1)],
                'cropend':[dt(2018, 1, 1), dt(2018, 1, 1), dt(2018, 1, 1)],
                'cropfile':['maizes', 'potatod', 'grassd'],
                'croptype':[1, 2, 3]})


crop = Crop(swcrop=True,
            rds=200.0,
            croprotation=croprotation)

## Irrigation

In [1]:
from pyswap.soilwater.irrigation import Irrigation, FixedIrrigation
from pandas import DataFrame

fixed_irrigation = FixedIrrigation(swirgfil=False,
                                   table_irrigevents=DataFrame({'time':[1.0, 360.0],
                                                            'irrigation':[20.0, 20.0]}))
                                   

# irrigation = Irrigation(schedule=False,
#                         swirfix=False)  # is this is set to true, then an error is returned if additional parameters are not given

In [2]:
from pprint import pprint

print(fixed_irrigation.model_string())

SWIRGFIL = False
 TIME  IRRIGATION
  1.0        20.0
360.0        20.0


## Drainage

In [30]:
from pyswap.soilwater.drainage import LateralDrainage

latdrain = LateralDrainage(swlatdrain=0)

ValidationError: 1 validation error for LateralDrainage
swdra
  Field required [type=missing, input_value={'swlatdrain': 0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing

TBD.

## Soil water

In [10]:
from pyswap.soilwater.soilmoisture import SoilMoisture
from pyswap.soilwater.surfaceflow import SurfaceFlow
from pyswap.soilwater.evaporation import Evaporation


soilmoisture = SoilMoisture(swinco=2,
                            gwli=-75.0)

surfaceflow = SurfaceFlow(swpondmx=False,
                          pondmx=0.2,
                          swrunon=False)

evaporation = Evaporation(swcfbs=False,
                          cfevappond=1.25,
                          swredu=1,
                          cofredbl=0.35,
                          rsigni=0.5)


In [11]:
from pyswap.soilwater.soilprofile import SoilProfile
from pandas import DataFrame

df_soilprofile = DataFrame({
    'ISUBLAY': [1, 2, 3, 4],
    'ISOILLAY': [1, 1, 2, 2],
    'HSUBLAY': [10.0, 20.0, 30.0, 140.0],
    'HCOMP': [1.0, 5.0, 5.0, 10.0],
    'NCOMP': [10, 4, 6, 14]
})

soilprofile = SoilProfile(soilprofile=df_soilprofile,
                          swsophy=False,
                          swhyst=0,
                          swmacro=False)



In [9]:
from pyswap.soilwater.snow import SnowAndFrost

snow = SnowAndFrost(swsnow=False,
                    swfrost=False)


In [14]:
from pyswap.soilwater.richards import RichardsSettings

richards = RichardsSettings(swkmean=1,
                            swkimpl=False)


In [1]:
from pyswap.core.boundary import LateralDrainage, BottomBoundary

lateral_drainage = LateralDrainage(swdra=0)
bottom_boundary = BottomBoundary(swbbcfile=False, 
                                 swbotb=6)

TypeError: typing.Annotated[bool | int, PlainSerializer(func=<function <lambda> at 0x7f846c9676a0>, return_type=<class 'int'>, when_used='json')] is not a generic class

In [3]:
lateral_drainage.model_string()

'SWDRA = 0\n'

## Putting the model together

In [16]:
from pyswap.core.model import Model


model = Model(metadata=meta,
            simsettings=simset,
            meteorology=meteo,
            crop=crop,
            irrigation=irrigation,
            soilmoisture=soilmoisture,
            surfaceflow=surfaceflow,
            evaporation=evaporation,
            soilprofile=soilprofile,
            snowandfrost=snow,
            richards=richards,
            lateraldrainage=lateral_drainage,
            bottomboundary=bottom_boundary)

In [25]:
# remove items from the model that are None
model.__dict__ = {k: v for k, v in model.__dict__.items() if v is not None}

for k, v in model.__dict__.items():
    print(v.model_string())

AUTHOR = John Doe
INSTITUTION = University of Somewhere
EMAIL = john.doe@somewhere.com
PROJECT_NAME = Test
SWAP_VER = 4.0
COMMENT = None
SWCROP = True
RDS = 200.0
CROPROTATION =     cropstart     cropend cropfile  croptype
0  2018-01-01  2018-01-01   maizes         1
1  2018-01-01  2018-01-01  potatod         2
2  2018-01-01  2018-01-01   grassd         3


AttributeError: 'Evaporation' object has no attribute 'model_string'